### Import libraries and data

In [ ]:
import pandas as pd
import csv
import re
import numpy as np
from scipy.optimize import nnls

In [ ]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [ ]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [ ]:
unitformula = pd.read_csv("scaledformulae.csv")
unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Calculations

In [ ]:
from sklearn import linear_model, svm
regVols = linear_model.Ridge(alpha=100)
regAlpha = linear_model.Ridge(alpha=100)

In [ ]:
def percentageCalc(predictedvolume,vols):
    percentage = predictedvolume.div(vols.Volume)
    for value in np.nditer(percentage):
        if (value >=0.95) and (value <=1.05):
            return 1
        else:
            return 0

In [ ]:
alpha = 0.00
x2=pd.DataFrame()
for step in range(1,50):
    print step
    regVols.fit(unitformula, vols.div(temps.Temperature*alpha+1, axis="index"))
    w = regVols.coef_[0]
    for line in zip(unitformula.columns.values, w):
        x2[line[0]] = unitformula[line[0]].apply(lambda x: x*line[1])
    x2["sum"] = x2.sum(axis=1)
    vol2 = vols.div(x2["sum"], axis="index") - 1
    regAlpha.fit(temps, vol2)
    alpha = regAlpha.coef_[0][0]
    unitformula["Keep"] = x2.apply(percCalc, args=(vols), axis=1)
    print "Element average volume (cubic angstroms)", zip(unitformula.columns.values,w)
    print "Alpha Value: ", alpha

Here we see that there is a new line of code; 
<pre>
unitformula["Keep"] = x2.apply(percCalc, args=(vols), axis=1)
</pre>
This is basically creating a new column where the entries are either decided to be kept or omitted for later iterations, by applying the percentage calculation function. This is an early test of it and the function is not being applied correctly.

In [ ]:
std =[]
std.append(w.std()/np.sqrt(len(w)))
for value in std:
    print value

A simple calculation of the standard deviation of the average element volumes